# Case

Naive sentiment analysis

This is a simple project using Naive Bayes Classifier and Scikit-learn to create a Google Play store reviews classifier (Sentiment Analysis) in Python. You will categorize user reviews as good or bad. The Naive Bayes classification technique is a simple and powerful classification task in machine learning. The Naive Bayes classification technique is a simple and powerful classification task in machine learning. In this dataset, we use the 23 most popular mobile apps, and only two columns.

Step 1:

We have three columns: package name, review and polarity (0 = bad, 1 = good) Preprocess the data by eliminating the package name column and putting all reviews in lower case.

Step 2:

Separate target from feature, and split your data.

Step 3:

Vectorize your features and use Naive Bayes to classify the reviews as good or bad. We will not focus on hypertuning our model this time. This was an introduction project to sentiment analysis using Naive Bayes.

Step 4:

Use the app.py to create your pipeline.

Save your naive bayes classification model in the 'models' folder.

Insall de libraries

In [125]:
!pip install pandas
!pip install numpy
!pip install sklearn
!pip install nltk
!pip install matplotlib
%pip install pickle


[notice] A new release of pip available: 22.2 -> 22.2.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2 -> 22.2.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2 -> 22.2.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2 -> 22.2.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2 -> 22.2.2
[notice] To update, run: pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle

[notice] A new release of pip available: 22.2 -> 22.2.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Imports

In [126]:
import pandas as pd
import numpy as np
import nltk #text processing
import re
import unicodedata
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB



In [127]:
#load the words to use as filter
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/gitpod/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Load Data of Reviews

In [128]:
url='https://raw.githubusercontent.com/4GeeksAcademy/naive-bayes-project-tutorial/main/playstore_reviews_dataset.csv'
df = pd.read_csv(url)

Explore Dataset

In [129]:
df.head()


,package_name,review,polarity
0,com.facebook.katana,privacy at least put some option appear offline. i mean for some people like me it's a big pressure to be seen online like you need to response on every message or else you be called seenzone only. if only i wanna do on facebook is to read on my newsfeed and just wanna response on message i want to. pls reconsidered my review. i tried to turn off chat but still can see me as online.,0
1,com.facebook.katana,"messenger issues ever since the last update, initial received messages don't get pushed to the messenger app and you don't get notification in the facebook app or messenger app. you open the facebook app and happen to see you have a message. you have to click the icon and it opens messenger. subsequent messages go through messenger app, unless you close the chat head... then you start over with no notification and having to go through the facebook app.",0
2,com.facebook.katana,profile any time my wife or anybody has more than one post and i view them it would take me to there profile so that i can view them all at once. now when i try to view them it tells me that the page that i requested is not available. i've restarted my phone i even cleard the cache and i've uninstalled and reinstalled the app and it is still doing it. please fix it thank you,0
3,com.facebook.katana,the new features suck for those of us who don't have a working back button can you guys make the videos able to be slid to the left to exit the video. as i have to force close facebook to exit,0
4,com.facebook.katana,"forced reload on uploading pic on replying comment last night i tried to reply a comment by uploading a photo from my phone. when i press on the button to select photos the app automatically goes back to the main page. on other occasions, i could enter to my gallery to select my image but as soon as i selected an image and press done, the program, again, forced reload and suddenly go back to the main page. please fix this and i will change my rating. thank you.",0


In [130]:
df['polarity'].value_counts()

0    584
1    307
Name: polarity, dtype: int64

In [131]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   package_name  891 non-null    object
 1   review        891 non-null    object
 2   polarity      891 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 21.0+ KB


Package name, review and polarity (0 = bad, 1 = good)

In [132]:
df.sample(10)

,package_name,review,polarity
321,com.viber.voip,no ringing tone whenever i call someone it shows that its ringing but has no calling sound. keeps going offline automatically . fix new update pls!,0
776,org.mozilla.firefox,"shab i have firefox on all of my devices and its more safer than the internet explorer. i like it i just wish it had more functions like free spyware, anti virus.",0
740,com.shirantech.kantipur,too much pop up add average app with too much advertisement. one star for the app and 5 star for the pop up advertisement.,0
812,org.mozilla.firefox,"gave u too many chances i love desktop firefox, this app is brutally annoying an cumbersome, i've installed it about 5 times thinking/hoping it would get better. there are much better browsers for android than this eg. dolphin or maxthon",1
99,com.linkedin.android,"updated version is down not able to sent connection request, no contacts synchronization . please develop soon",0
443,com.whatsapp,few ui updates 1. group should be included with group status... so that if common event is there then the event can be posted through group status.. and group name need not be changed. . . as it becomes difficult to identify a particular group if a particular festival is there and all group name seems to be similar. 2. profile picture and profile status should be at one place not seperate... for status we have shortcut but for profile picture we have to go into settings then profile then comes profile picture.,0
222,com.supercell.clashofclans,"this update sucks!!! i can't open the game anymore, just crashed. damn it! i have an on going war but the app won't run... if there is a half star option to rate this, i would give you that. fix this asap....",0
493,com.Slack,wonderful for project coordination this is a great tool to have when coordinating projects. it keeps everyone on the same page and in communication.,1
852,com.rovio.angrybirds,its cool my brother doesnt bug m me anymore i show him angry birds and his mouth shuts but some levels are almost impossible and i have to wait for free power ups or buy them,1
509,com.Slack,"free the gifs not reliable on a slower network, messages frequently don't send",0



Text Processing - Clean

Step 1 - change all to lower case


In [133]:
#Preprocess the data by eliminating the package name column and putting all reviews in lower case.

df.drop(['package_name'],axis=1, inplace=True)
#chante all to lower case
df['review'] = df['review'].str.lower()




Step 2 - Remove all words that not have meaning to the search - stop words


In [134]:
#see, the word in the reviews, create a list of word and frecuency
df['review'].str.split(expand = True).stack().value_counts()[:60] #(show top 60)

the       1293
to        1159
i         1084
and        853
it         778
a          619
is         555
my         442
this       421
for        410
of         390
but        348
in         340
on         325
not        318
app        317
you        315
that       260
have       252
with       220
so         203
be         194
no         186
when       182
if         180
or         169
all        163
can        158
as         148
me         145
are        139
it's       138
can't      137
update     136
new        136
like       135
fix        133
please     131
good       128
game       121
one        119
use        118
just       117
get        112
its        111
very       109
more       107
was        104
there      104
don't      103
now        102
will       102
even       101
from       101
at         100
up          99
great       99
time        93
your        90
do          89
dtype: int64

In [135]:
#remove the stopword
stop = stopwords.words('english')

def remove_stopwords(text):
  if text is not None:
    #list of the word in the text
    words = text.strip().split()
    words_filtered = []
    for word in words:
      if word not in stop:
        words_filtered.append(word)
    result = " ".join(words_filtered) #join the word in a text with space separation
  else:
      result = None
  return result




Eliminamos los caracteres desconocidos


In [136]:
# elimina espacio libre al principio y al final
df['review'] = df['review'].str.strip()
df['review'] = df['review'].str.lower()


In [137]:
df['review']


0                                                                                      privacy at least put some option appear offline. i mean for some people like me it's a big pressure to be seen online like you need to response on every message or else you be called seenzone only. if only i wanna do on facebook is to read on my newsfeed and just wanna response on message i want to. pls reconsidered my review. i tried to turn off chat but still can see me as online.
1               messenger issues ever since the last update, initial received messages don't get pushed to the messenger app and you don't get notification in the facebook app or messenger app. you open the facebook app and happen to see you have a message. you have to click the icon and it opens messenger. subsequent messages go through messenger app, unless you close the chat head... then you start over with no notification and having to go through the facebook app.
2                                                     

In [138]:
def normalize_str(text_string):
    if text_string is not None:
        result=unicodedata.normalize('NFD',text_string).encode('ascii','ignore').decode()
    else:
        result=None 
    return result

In [139]:
df['review']=df['review'].apply(normalize_str)
df['review']=df['review'].str.replace('!','')
df['review']=df['review'].str.replace(',','')
df['review']=df['review'].str.replace('&','')
df['review']=df['review'].str.normalize('NFKC')
df['review']=df['review'].str.replace(r'([a-zA-Z])\1{2,}',r'\1',regex=True)

In [140]:
df.sample(10)

,review,polarity
741,it is best app for regular news update but ...please add more news updates in nepali language too.,1
234,personal break don't mind the break necessarily but i don't think its necessary. especially during gem boost time. would like to not be told when i can't play lol.,0
195,nice game.... its simply amazing...but i would like it better if we had custom options for graphics. it runs extremely smooth but the developer should go easy on the memory that is it takes more space now than ever before. but overall its an epic game.,1
197,would give 5* if i could retrieve my previous game my phone crashed and i had to install it again didn't get to keep my score coins and boosts the gems i paid for where also lost negatives aside i enjoy this game its a good time waster i thinks it's better than the first temple run.,0
623,i love it i love using uc browser mini because browsing speed of it is marvelous and mind blowing and the size of this app is so small. so overall i would like to say that size of it is small but the work of it is big and awesome.,1
629,all in one app dis version of uc browser mini disappointed me as search showing in notification bar. and when v download anything was not visble in notification bar as previous version. its request to look into it and try to solve it otherwise as i have to uninstall it and goto some other app.,0
244,this country not allowed? i would have given it 5 stars because my searches and browsing are spot on but it took me to back up account and when i put my backup e-mail in it said this country not allowed: im in america wtf????,0
417,doesn't work anymore before the last update it worked just fine on my tablet but now it will start to open then close down.,0
782,it was good but the last update caused so many forced close. it was good but the last update caused so many forced close. now i can't really enjoy my browsing experience. thanks for nothing,0
570,doesn't work won't allow you to log in.,0


In [141]:
df.iloc[675,]

review      loved it rzrl app rrlrrl rrsrzrlrrl r rrsrzrlrrl r rr rsrrz rzrrzrz rrlrsrl rrlrrz rlrsrl rr rzrrl rrsrzrlrrl rsrlrrl rlrzrzrl rlrrlr inserting symbols inserting pictures from the galary wedding 123 etc. and phonetic symbols so the english learners could be helped more. rrsrz rlrrl rrlrrz rrz rsrzrrzrz rrlrsrlrz rsrrz rzrl  rrtrlrrz rsrlrrzrlr rlrrzrsrrlrrzr rrsrzrlr rlrrl r rl
polarity                                                                                                                                                                                                                                                                                                                                                                                               1
Name: 675, dtype: object

In [142]:
df_interim = df.copy()

In [143]:
df_interim['review']=df_interim['review'].str.lower()

In [144]:
df_interim['review'] = df_interim['review'].apply(remove_stopwords)


In [145]:
df_interim['review'].str.split(expand=True).stack().value_counts()[:60] 

app         351
update      150
game        140
please      138
like        137
can't       137
new         137
fix         135
good        134
use         123
one         120
great       115
get         112
even        102
time         98
browser      85
really       84
back         81
still        80
would        80
love         77
make         74
5            71
work         71
also         69
see          69
using        68
phone        68
i'm          67
open         66
it.          65
want         64
google       64
much         62
every        61
version      60
option       59
u            58
way          57
app.         56
go           56
android      56
used         55
best         54
download     53
better       53
play         52
.            51
give         50
send         49
people       48
able         48
message      47
since        47
many         46
works        45
apps         45
ads          45
keep         45
old          44
dtype: int64

In [146]:
def word_only_letters(word):
    for c in word:
        cat = unicodedata.category(c)
        if cat not in ('Ll','Lu'):  #only letter upper y lower
            return False
    return True

# clean only letter
def text_only_letters(text):
    if text is not None:
        #list of the word in the text
        words = text.strip().split()
        words_filtered = []
        for word in words:
            if word_only_letters(word):
                words_filtered.append(word)
            result = " ".join(words_filtered) #join the word in a text with space separation
    else:
        result = None
    return result

In [147]:
df_interim['review'] = df_interim['review'].apply(text_only_letters)

In [148]:
#remove multi letter looove iiiitttt, or repeat secuence
def replace_multiple_letters(message):
  if message is not None:
    result = re.sub(r'(.+?)\1+', r'\1', message)
    #result = re.sub(r"([a-zA-Z])＼1{2,}", r"＼1", message)
  else:
    result = None
  return result


In [149]:
df_interim['review'] = df_interim['review'].apply(replace_multiple_letters)

In [150]:
df_interim['review']

0                                                                           privacy least put option apear mean people like big presure sen online like ned response every mesage else caled senzone wana facebok read newsfed wana response mesage want pls reconsidered tried turn chat stil se
1                                           mesenger isues ever since last update initial received mesages get pushed mesenger ap get notification facebok ap mesenger open facebok ap hapen se click icon opens subsequent mesages go mesenger ap unles close chat start notification go facebok
2                                                                                                                   profile time wife anybody one post view would take profile view try view tels page requested restarted phone even cleard cache uninstaled reinstaled ap stil please fix thank
3                                                                                                                                 

In [151]:
#copy to df
df = df_interim.copy()




Setp 3 - Model


In [152]:
# Separate target and predictor
X = df['review']
y = df['polarity']


In [153]:



vectorizer = CountVectorizer(stop_words='english')
X = vectorizer.fit_transform(X)
vectorizer.get_feature_names_out()



array(['abesta', 'abilities', 'ability', ..., 'zespole', 'zom', 'zoming'],
      dtype=object)

In [154]:
### aquí guardo el modelo para usar en la app?
filename = '../models/vectorizer.pkl' #use absolute path
pickle.dump(vectorizer, open(filename, 'wb'))


In [155]:
X = X.toarray()

In [156]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=25)

In [157]:
#Vectorizer change matriz with 0-1
#vec = CountVectorizer(stop_words = "english")
#X_train = vec.fit_transform(X_train).toarray()
#X_test = vec.transform(X_test).toarray()

In [158]:
print(X_train)


[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [159]:
print(X.shape)
print(X_test.shape)
print(X_train.shape)

(891, 3289)
(223, 3289)
(668, 3289)


In [160]:
clf = GaussianNB()
clf.fit(X_train, y_train)

GaussianNB()

In [161]:
y_train.value_counts()

0    438
1    230
Name: polarity, dtype: int64

In [162]:
y_train_pred = clf.predict(X_train)
y_test_pred = clf.predict(X_test)

In [163]:
target_names = ['bad', 'good']
print(classification_report(y_train, y_train_pred, target_names=target_names))

              precision    recall  f1-score   support

         bad       1.00      0.98      0.99       438
        good       0.96      1.00      0.98       230

    accuracy                           0.99       668
   macro avg       0.98      0.99      0.99       668
weighted avg       0.99      0.99      0.99       668



In [164]:
print(classification_report(y_test, y_test_pred, target_names=target_names))


              precision    recall  f1-score   support

         bad       0.80      0.83      0.81       146
        good       0.65      0.61      0.63        77

    accuracy                           0.75       223
   macro avg       0.73      0.72      0.72       223
weighted avg       0.75      0.75      0.75       223



No resultó muy bueno - overfit

# Naive Bayes

Naive Bayes is a probabilistic algorithm for classification, based on Bayes Theorem. This algorithm really makes you understand a big deal about classification and it is probably one of the most useful algorithms for simple Machine Learning use cases. It is simple enough to be completed by hand.

In [165]:
clf_multinomial = MultinomialNB()
clf_multinomial.fit(X_train, y_train)

MultinomialNB()

In [166]:
MultinomialNB()


MultinomialNB()

In [167]:
y_train_pred = clf_multinomial.predict(X_train)
y_test_pred = clf_multinomial.predict(X_test)


In [168]:
print(classification_report(y_train, y_train_pred, target_names=target_names))


              precision    recall  f1-score   support

         bad       0.95      0.97      0.96       438
        good       0.95      0.90      0.92       230

    accuracy                           0.95       668
   macro avg       0.95      0.94      0.94       668
weighted avg       0.95      0.95      0.95       668



In [169]:
print(classification_report(y_test, y_test_pred, target_names=target_names))


              precision    recall  f1-score   support

         bad       0.86      0.90      0.88       146
        good       0.79      0.71      0.75        77

    accuracy                           0.83       223
   macro avg       0.82      0.81      0.81       223
weighted avg       0.83      0.83      0.83       223



In [170]:
Z = vectorizer.transform(['I like this app'])

In [171]:
print(np.array(Z))
print(Z.shape)
print(Z)

  (0, 142)	1
  (0, 1602)	1
(1, 3289)
  (0, 142)	1
  (0, 1602)	1


In [172]:
# Modelo GaussianNB
print(clf.predict(Z.toarray()))



[1]


In [173]:
# Modelo Multinomial
print(clf_multinomial.predict(Z.toarray()))


[1]


In [174]:
pd.set_option('display.max_colwidth', None)
df.tail(10)

,review,polarity
881,game ruined felt like playing awhile imediately second awful mobile game ad poped second level loking store realized pay remove yep,0
882,ads way many ads even enjoy game amount ads every level,0
883,great game many ads almost worth,1
884,fun hard angry birds realy fun video game people least year olds people lot years older one really chalenge teach children control even though ba litle,1
885,many ads far adverts game know fre ned ads make profit neds,1
886,loved incredible awesome go power make new clash clans thing but beter,1
887,time legendary game birthday party levels short fuse levels pigs crash onto diferent chemicals players cringe much ads close ur conection play ads trouble,1
888,ads way heavy listen bad ads every round whether pas fail sometimes ads next round starts spend time ads game develop web aps honestly many people rely ads make apreciate however developers went far beyond frankly disrespectful,0
889,fun works perfectly ads anoying think especialy fre,1
890,everywhere se angry birds everywhere stop playing get head devs perfect,1


In [175]:
Z2 = vectorizer.transform(['I do not like this app'])

# Modelo GaussianNB
print(f"Gauss: {clf.predict(Z2.toarray())}")

# Modelo Multinomial
print(f"Multi: {clf_multinomial.predict(Z2.toarray())}")



Gauss: [1]
Multi: [1]


Guardar Modelo

In [176]:
pickle.dump(clf_multinomial, open('../models/multinomial.pkl', 'wb'))

Traer el Modelo

In [177]:
load_model = pickle.load(open('../models/multinomial.pkl', 'rb'))


In [178]:
Z2 = vectorizer.transform(['I do not like this app'])
# Modelo Multinomial loaded
print(f"Multi: {load_model.predict(Z2.toarray())}")


Multi: [1]


In [179]:
!pip install torch
!pip install tensorflow
!pip install -q transformers

from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")


[notice] A new release of pip available: 22.2 -> 22.2.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2 -> 22.2.2
[notice] To update, run: pip install --upgrade pip


No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [180]:
data = ["I love you", "I hate you"]
sentiment_pipeline(data)

[{'label': 'POSITIVE', 'score': 0.9998656511306763},
 {'label': 'NEGATIVE', 'score': 0.9991129040718079}]

In [181]:
##nb.predict(vec.transform(['i don not like','eat much ram','eat','like']))


In [182]:
##nb.predict(vec.transform(['love','great solid love important feature','ram','like fast']))

In [183]:
df[df['polarity']==1]['review'].sample(10)

836                                                                                                                                                                                                                                                                    useful others ap like
182                                                                                                                                                                                                                                                      prety god say oh wait fun game play
581                                                                                                                                                                                           useful used much could even evernote one favorite aps kep making beter widgets make even coler
577                                                                                                                                              

In [184]:
df[df['polarity']==0]['review'].sample(10)

389                                                                                                                                                                                                                      stil come used use unaceptable instead ading new welcome scren latest update brought back widget
737                                                                                                                                                                                                                                                 god download one cal apliation start browsing sped slow founder solve
250                                                                                                                                                                                                              canot cast chromecast aps chromecast suports chrome browser suposed cast use worse wait fix already come
771                                                       

Save the model 


In [185]:
#filename = '../modelo_final.sav' #use absolute path
#pickle.dump(nb, open('../models/modelo_final.sav', 'wb'))
